# _init_repoyard

In [ ]:
#|default_exp cmds._init_repoyard
#|export_as_func true

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()

Environment variable DISABLE_NBLITE_EXPORT is set to True, skipping export.


In [ ]:
#|top_export
from pathlib import Path

from repoyard import const

In [ ]:
#|set_func_signature
def init_repoyard(
    config_path: Path|None = None,
    data_path: Path|None = None,
    verbose: bool = False,
):
    """
    Initialize repoyard.
    
    Will create the necessary folders and files to start using repoyard.
    """
    ...

Set up testing args

In [ ]:
from tests.utils import *
remote_name, remote_rclone_path, config, config_path, data_path = create_repoyards()

In [ ]:
# Args
config_path = config_path
data_path = data_path
verbose = True

# Function body

In [ ]:
#|export
config_path = config_path or const.DEFAULT_CONFIG_PATH
data_path = data_path or const.DEFAULT_DATA_PATH

if config_path.expanduser().as_posix() != const.DEFAULT_CONFIG_PATH.expanduser().as_posix():
    if verbose: print(f"Using a non-default config path. Please set the environment variable {const.ENV_VAR_REPOYARD_CONFIG_PATH} to the given config path for repoyard to use it. ")

Using a non-default config path. Please set the environment variable REPOYARD_CONFIG_PATH to the given config path for repoyard to use it. 


Create a default config file if it doesn't exist

In [ ]:
#|export
from repoyard.config import get_config, _get_default_config_dict, Config
import toml
if not config_path.expanduser().exists():
    if verbose: print("Creating config file at:", config_path)
    Path(config_path).expanduser().parent.mkdir(parents=True, exist_ok=True)
    default_config_dict = _get_default_config_dict(config_path=config_path, data_path=data_path)
    del default_config_dict['config_path'] # Don't save the config path to the config file
    config_toml = toml.dumps(default_config_dict)
    
    Path(config_path).expanduser().write_text(config_toml)
config = get_config(config_path)

Create the default `.rclone_exclude` file

In [ ]:
#|export
if not config.default_rclone_exclude_path.exists():
    config.default_rclone_exclude_path.write_text(const.DEFAULT_RCLONE_EXCLUDE)

For testing purposes, modify the config

In [ ]:
config = Config(**{
    'config_path' : config_path,
    **config.model_dump(),
    'storage_locations' : {
        'fake' : {
            'storage_type' : 'rclone',
            'store_path' : data_path / "fake_store",
        }
    }
})

Create folders

In [ ]:
#|export
paths = [
    config.repoyard_data_path,
    config.local_store_path,
]

for path in paths:
    if not path.exists():
        if verbose: print(f"Creating folder: {path}")
        path.mkdir(parents=True, exist_ok=True)

Set up symlinks for every storage location of type `local`

In [ ]:
#|export
from repoyard.config import StorageType
for storage_location_name, storage_location in config.storage_locations.items():
    if storage_location.storage_type != StorageType.LOCAL.value: continue
    storage_location.store_path.mkdir(parents=True, exist_ok=True)
    if (config.local_store_path / storage_location_name).exists():
        (config.local_store_path / storage_location_name).unlink()
    (config.local_store_path / storage_location_name).symlink_to(storage_location.store_path)

Create `repoyard_rclone.conf` if it doesn't exist

In [ ]:
#|export
from repoyard.config import _default_rclone_config
if not config.rclone_config_path.exists():
    if verbose: print(f"Creating rclone config file at: {config.rclone_config_path}")
    config.rclone_config_path.write_text(_default_rclone_config)

Done

In [ ]:
#|export
if verbose:
    print("Done!\n")
    print("You can modify the config at:", config_path)
    print("All repoyard data is stored in:", config.repoyard_data_path)

Done!

You can modify the config at: /tmp/repoyard_0_n_0vtox6/.config/repoyard/config.toml
All repoyard data is stored in: /tmp/repoyard_0_n_0vtox6/.repoyard
